## About

`auto EDA for time-series` は探索的データ分析 (EDA) を省力化するためのツールです

できること

以下の一連の手続きをJupyter Notebook上で手早く分析することができます

- データの初回分析としての可視化
- 可視化を受けてのデータクレンジング
- クレンジング後の結果確認といった一連の手続き


    Environment: Python3.6 (Anaconda)
    Requirement (python): numpy, pandas, matplotlib, seaborn, scikit-learn
    Requirement (jupyter): jupyter_contrib_nbextensions (Table of Content)
    Author: FRI) H.Shibata
    Last update: 2018.10.16

## How to use

1. Modify cells in data preparation section (path of `read_csv()`, name of `time_column` and `obj_column`)
2. Execute `Cell -> Run All`
3. Execite `File -> Download as -> HTML`

## Preparation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from mylib.my_module import *
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.notebook_repr_html = True
pd.options.display.precision = 3

## Data preparation

### Import

    デフォルトではpd.read_csv()をするだけ
    オプションが必要な場合、セル内のコードを変更すること

In [ ]:
# change args of read_csv depending on your data
df = pd.read_csv("./data/ozone.csv", na_values="?")
display(df.shape)
df.head()

### Define column

    目的変数および時系列参照用のカラムの定義
    データに応じてカラム名を変更すること

In [ ]:
obj_column = "Ozone"
time_column = "Date"
df[[time_column]] = pd.to_datetime(df[time_column]) # 時系列のカラムが日時型のデータの場合にコメントアウト

### Cleansing

    必要に応じてデータ加工スクリプトを以下のセルに書く
    
    例
    - 基礎分析に不要なカラムの除外 (ID列とか)
    - 0/1や1~5のintなどのデータを数値からカテゴリに変換する (df[col]=df[col].astype("object"))
    - 以降の分析結果をフィードバックして再分析したい場合

In [ ]:
#

## Chronological change
  
    データの時系列的な変化の可視化
    同じ時点のデータが複数ある場合、平均値をプロットする

In [ ]:
col_num = df.select_dtypes("number").columns.ravel()
outdir = "output/fig/timeseries"
mkdir(outdir)
for i, name in enumerate(col_num):
    fig = df.groupby(time_column).agg({name:"mean"}).plot(y=name, kind="line", title=name)
    plt.savefig(outdir+"/"+name+".png")
    plt.show()
    plt.close('all')

## Basic stats summary
  
    各カラムの基本統計量
    共通：値の数、NA数
    時間：最初と最後
    数値：平均、標準偏差、最小値、25%ile, ..., 最大値
    カテゴリ：最頻値、最頻値のカウント

In [ ]:
desc = df.describe(include='all')
desc.loc["NA_count"] = (df.isna().sum())
desc = desc.T
outdir = "output/csv"
mkdir(outdir)
desc.to_csv(outdir+"/stats_summary.csv")
desc

## Distribution of each variable

### Numeric variables

    数値データの分布

In [ ]:
col_num = df.select_dtypes("number").columns.ravel()
outdir = "output/fig/hist"
mkdir(outdir)
for name in col_num:
    fig = df[name].plot("hist", title=name)
    plt.savefig(outdir+"/"+name+".png")
    plt.show()
    plt.close('all')

### Categorical variables

    カテゴリカルデータの分布

In [ ]:
col_cate = df.select_dtypes("object").columns.ravel()
outdir = "output/fig/categorical"
mkdir(outdir)
for i, name in enumerate(col_cate):
    fig = df[name].value_counts().plot("bar", title=name)
    plt.savefig(outdir+"/"+name+".png")
    plt.show()
    plt.close('all')

## Correlation among numeric variables

    変数間の相関を可視化する

### Correlation matrix

    ピアソンの相関係数の行列

In [ ]:
num_col = df.select_dtypes("number").columns.ravel()
cor = df.loc[:, num_col].corr()
display("Corelation matrix")
display(cor)
outdir = "output/csv"
cor.to_csv(outdir+"/corr.csv")

### Corelation heatmap

    相関行列のヒートマップ表現

In [ ]:
display("Corelation heatmap")
plt.figure(figsize=(24, 24))
sns_plot = sns.heatmap(cor)
outdir = "output/fig"
plt.savefig(outdir+"/corplot.png")

### Variable pairs which is highly corelated

    相関係数の高い変数の組の一覧 (デフォルトは0.95。必要に応じてセル内の値を変更すること)

In [ ]:
cor_threshold = 0.95
cor_tri = pd.DataFrame(np.triu(cor.values, k=1), index=num_col, columns=num_col)
corpair = pd.DataFrame({
    "var1": cor_tri.columns.ravel()[np.where(cor_tri.values > cor_threshold)[0]],
    "var2": cor_tri.columns.ravel()[np.where(cor_tri.values > cor_threshold)[1]],
    "cor": cor_tri.values[np.where(cor_tri.values > cor_threshold)],
}, columns = ["var1", "var2", "cor"]
)
display(corpair)
outdir = "output/csv"
corpair.to_csv(outdir+"/pair_cor_"+str(cor_threshold)+".csv", index=False)

## Data topology

    データを2次元空間にプロットし、データのばらつきを可視化する
    カテゴリカルなデータはダミー展開する (極端にカテゴリ数が多い場合、時間がかかる)
    欠損値は列方向で中央値補完 (全行欠損の列はまるごと削除)

### PCA

    PCAの上位2成分による次元圧縮

In [ ]:
X = pd.get_dummies(df.fillna(df.mean()).dropna(axis=1).drop([obj_column, time_column], axis=1)).values
y = df[obj_column].values
pca_res = PCA(n_components=2).fit_transform(X)
sns.scatterplot(x=pca_res[:,0], y=pca_res[:,1], hue=y)
outdir = "output/fig"
plt.savefig(outdir+"/pca.png")

### t-SNE

    非線形変換（データ点同士の類似度を保つ次元削減）による次元圧縮
    レコード数が多いと計算時間がかかるので、サクッと見たい場合は削除すること
    (目安として10万行200列で2h程度、20万行100列で8h程度)

In [ ]:
tsne_res = TSNE(n_components=2, random_state=0).fit_transform(X)
sns.scatterplot(x=tsne_res[:,0], y=tsne_res[:,1], hue=y)
outdir = "output/fig"
plt.savefig(outdir+"/tsne.png")